In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib
import warnings
import sklearn

In [22]:
from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all" 
InteractiveShell.ast_node_interactivity = "last_expr" 

In [3]:
#import gensim
import scipy
import json
import nltk
import sys
import csv
import os

In [4]:
print('matplotlib: {}'.format(matplotlib.__version__))
print('scipy: {}'.format(scipy.__version__))
print('seaborn: {}'.format(sns.__version__))
print('pandas: {}'.format(pd.__version__))
print('numpy: {}'.format(np.__version__))
print('Python: {}'.format(sys.version))

matplotlib: 2.0.2
scipy: 1.2.1
seaborn: 0.7.1
pandas: 0.20.1
numpy: 1.16.3
Python: 3.6.8 |Anaconda custom (64-bit)| (default, Dec 29 2018, 19:04:46) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


In [4]:
sns.set(style='white', context='notebook', palette='deep')
#warnings.filterwarnings('ignore')
sns.set_style('white')
%matplotlib inline

### Fetching the data

In [5]:
print(os.listdir("input/"))

['.DS_Store', 'test_stage_2.tsv', 'test_stage_1.tsv', 'sample_submission_stage_1.csv', 'sample_submission_stage_2.csv']


In [6]:
gendered_pronoun_df = pd.read_csv('input/test_stage_1.tsv', delimiter='\t')

In [7]:
submission = pd.read_csv('input/sample_submission_stage_1.csv')

In [8]:
gendered_pronoun_df.shape

(2000, 9)

In [9]:
submission.shape

(2000, 4)

In [10]:
gendered_pronoun_df.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,B,B-offset,URL
0,development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,Pauline,207,http://en.wikipedia.org/wiki/List_of_Teachers_...
1,development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,Bernard Leach,251,http://en.wikipedia.org/wiki/Warren_MacKenzie
2,development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,De la Sota,246,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_...
3,development-4,The current members of Crime have also perform...,his,321,Hell,174,Henry Rosenthal,336,http://en.wikipedia.org/wiki/Crime_(band)
4,development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,Rivera,294,http://en.wikipedia.org/wiki/Jessica_Rivera


In [11]:
gendered_pronoun_df.Text[2]

"He had been reelected to Congress, but resigned in 1990 to accept a post as Ambassador to Brazil. De la Sota again ran for governor of C*rdoba in 1991. Defeated by Governor Angeloz by over 15%, this latter setback was significant because it cost De la Sota much of his support within the Justicialist Party (which was flush with victory in the 1991 mid-terms), leading to President Carlos Menem 's endorsement of a separate party list in C*rdoba for the 1993 mid-term elections, and to De la Sota's failure to regain a seat in Congress."

In [12]:
gendered_pronoun_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 9 columns):
ID                2000 non-null object
Text              2000 non-null object
Pronoun           2000 non-null object
Pronoun-offset    2000 non-null int64
A                 2000 non-null object
A-offset          2000 non-null int64
B                 2000 non-null object
B-offset          2000 non-null int64
URL               2000 non-null object
dtypes: int64(3), object(6)
memory usage: 140.7+ KB


In [13]:
train_df = gendered_pronoun_df.drop(['URL'], axis=1)

In [14]:
train_df.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,B,B-offset
0,development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,Pauline,207
1,development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,Bernard Leach,251
2,development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,De la Sota,246
3,development-4,The current members of Crime have also perform...,his,321,Hell,174,Henry Rosenthal,336
4,development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,Rivera,294


 <font color='blue'>**The goal is to give the probabilities for the 'Pronoun' in 'Text' to refer to 'A' or 'B' or 'Neither'.
The train data is not labeled in this challenge.**</font>

### New Features
(We will end up not using these features. This is here just just to play with the data.)

We make use of the (pedagogical) kaggle kernel https://www.kaggle.com/mjbahmani/top-3-nlp-libraries-tutorial-nltk-spacy-gensim .

**Number of words**

In [52]:
train_df["num_words"] = train_df["Text"].apply(lambda x: len(str(x).split()))

In [58]:
print('maximum of num_words in data_df',train_df["num_words"].max())
print('min of num_words in data_df',train_df["num_words"].min())

maximum of num_words in data_df 204
min of num_words in data_df 16


#### Number of unique words

In [59]:
train_df["num_unique_words"] = train_df["Text"].apply(lambda x: len(set(str(x).split())))
print('maximum of num_unique_words in train',train_df["num_unique_words"].max())
print('mean of num_unique_words in data_df',train_df["num_unique_words"].mean())

maximum of num_unique_words in train 136
mean of num_unique_words in data_df 56.4005


#### Number of characters

In [61]:
train_df["num_chars"] = train_df["Text"].apply(lambda x: len(str(x)))
print('maximum of num_chars in data_df',train_df["num_chars"].max())

maximum of num_chars in data_df 1270


#### Number of stopwords

In [64]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Benjamin/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [65]:
from nltk.corpus import stopwords
eng_stopwords = set(stopwords.words("english"))

In [69]:
eng_stopwords

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [70]:
train_df["num_stopwords"] = train_df["Text"].apply(lambda x: len([w for w in str(x).lower().split() 
                                                                  if w in eng_stopwords]))

print('maximum of num_stopwords in data_df',train_df["num_stopwords"].max())

maximum of num_stopwords in data_df 88


**Number of punctuations**

In [71]:
import string

train_df["num_punctuations"] =train_df['Text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )
print('maximum of num_punctuations in data_df',train_df["num_punctuations"].max())

maximum of num_punctuations in data_df 93


**Number of title case words**

In [72]:
train_df["num_words_upper"] = train_df["Text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
print('maximum of num_words_upper in data_df',train_df["num_words_upper"].max())

maximum of num_words_upper in data_df 11


In [74]:
print(train_df.columns)
train_df.head(3)

Index(['ID', 'Text', 'Pronoun', 'Pronoun-offset', 'A', 'A-offset', 'B',
       'B-offset', 'num_words', 'num_unique_words', 'num_chars',
       'num_stopwords', 'num_punctuations', 'num_words_upper'],
      dtype='object')


,ID,Text,Pronoun,Pronoun-offset,A,A-offset,B,B-offset,num_words,num_unique_words,num_chars,num_stopwords,num_punctuations,num_words_upper
0,development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,Pauline,207,74,61,426,29,14,0
1,development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,Bernard Leach,251,65,58,410,22,12,0
2,development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,De la Sota,246,95,71,536,39,16,0


#### Pronoun binaries

In [76]:
pronoun = train_df.Pronoun

In [77]:
np.unique(pronoun)

array(['He', 'Her', 'His', 'She', 'he', 'her', 'him', 'his', 'she'],
      dtype=object)

In [158]:
binary = {
    "He": 0,
    "he": 0,
    "She": 1,
    "she": 1,
    "His": 2,
    "his": 2,
    "Him": 3,
    "him": 3,
    "Her": 4,
    "her": 4
}
for index in range(len(train_df)):
    key = train_df.iloc[index]['Pronoun']
    train_df.at[index, 'Pronoun_binary'] = binary[key]
train_df.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,B,B-offset,num_words,num_unique_words,num_chars,num_stopwords,num_punctuations,num_words_upper,Pronoun_binary
0,development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,Pauline,207,74,61,426,29,14,0,4.0
1,development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,Bernard Leach,251,65,58,410,22,12,0,2.0
2,development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,De la Sota,246,95,71,536,39,16,0,2.0
3,development-4,The current members of Crime have also perform...,his,321,Hell,174,Henry Rosenthal,336,69,58,401,24,13,1,2.0
4,development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,Rivera,294,112,80,660,43,18,1,1.0


### Matei Ionita's kernel  (using BERT embeddings)
https://www.kaggle.com/mateiionita/taming-the-bert-a-baseline

Here we use the GAP files, which contain the training labels of the dataset (and the datasets).  
Then we use BERT's (contextual) embeddings for the three words A, B and P, and we train a multi-layer perceptron model (with Keras).

In [5]:
import os
import gc
import numpy as np
import pandas as pd 
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

In [2]:
import wget

In [187]:
wget.download('https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip')

'uncased_L-12_H-768_A-12.zip'

In [190]:
!unzip uncased_L-12_H-768_A-12.zip

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [184]:
wget.download('https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-development.tsv')
wget.download('https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-validation.tsv')
wget.download('https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-test.tsv')
wget.download('https://raw.githubusercontent.com/google-research/bert/master/modeling.py')
wget.download('https://raw.githubusercontent.com/google-research/bert/master/extract_features.py')
wget.download('https://raw.githubusercontent.com/google-research/bert/master/tokenization.py')

'tokenization.py'

In [6]:
import modeling
import extract_features
import tokenization

In [7]:
def compute_offset_no_spaces(text, offset):
    count = 0
    for pos in range(offset):
        if text[pos] != " ": 
            count +=1
    return count

def count_chars_no_special(text):
    count = 0
    special_char_list = ["#"]
    for pos in range(len(text)):
        if text[pos] not in special_char_list: 
            count +=1
    return count

def count_length_no_special(text):
    count = 0
    special_char_list = ["#", " "]
    for pos in range(len(text)):
        if text[pos] not in special_char_list: 
            count +=1
    return count

In [9]:
def run_bert(data):
    '''
    Runs a forward propagation of BERT on input text, extracting contextual word embeddings
    Input: data, a pandas DataFrame containing the information in one of the GAP files

    Output: emb, a pandas DataFrame containing contextual embeddings for the words A, B and Pronoun. Each embedding is a numpy array of shape (768)
    columns: "emb_A": the embedding for word A
             "emb_B": the embedding for word B
             "emb_P": the embedding for the pronoun
             "label": the answer to the coreference problem: "A", "B" or "NEITHER"
    '''
    # From the current file, take the text only, and write it in a file which will be passed to BERT
    text = data["Text"]
    text.to_csv("input.txt", index = False, header = False)

    # The script extract_features.py runs forward propagation through BERT, and writes the output in the file output.jsonl
    # I'm lazy, so I'm only saving the output of the last layer. Feel free to change --layers = -1 to save the output of other layers.
    os.system("python3 extract_features.py \
      --input_file=input.txt \
      --output_file=output.jsonl \
      --vocab_file=uncased_L-12_H-768_A-12/vocab.txt \
      --bert_config_file=uncased_L-12_H-768_A-12/bert_config.json \
      --init_checkpoint=uncased_L-12_H-768_A-12/bert_model.ckpt \
      --layers=-1 \
      --max_seq_length=256 \
      --batch_size=8")

    bert_output = pd.read_json("output.jsonl", lines = True)

    os.system("rm output.jsonl")
    os.system("rm input.txt")

    index = data.index
    columns = ["emb_A", "emb_B", "emb_P", "label"]
    emb = pd.DataFrame(index = index, columns = columns)
    emb.index.name = "ID"

    for i in range(len(data)): # For each line in the data file
        # get the words A, B, Pronoun. Convert them to lower case, since we're using the uncased version of BERT
        P = data.loc[i,"Pronoun"].lower()
        A = data.loc[i,"A"].lower()
        B = data.loc[i,"B"].lower()

        # For each word, find the offset not counting spaces. This is necessary for comparison with the output of BERT
        P_offset = compute_offset_no_spaces(data.loc[i,"Text"], data.loc[i,"Pronoun-offset"])
        A_offset = compute_offset_no_spaces(data.loc[i,"Text"], data.loc[i,"A-offset"])
        B_offset = compute_offset_no_spaces(data.loc[i,"Text"], data.loc[i,"B-offset"])
        # Figure out the length of A, B, not counting spaces or special characters
        A_length = count_length_no_special(A)
        B_length = count_length_no_special(B)

        # Initialize embeddings with zeros
        emb_A = np.zeros(768)
        emb_B = np.zeros(768)
        emb_P = np.zeros(768)

        # Initialize counts
        count_chars = 0
        cnt_A, cnt_B, cnt_P = 0, 0, 0

        features = pd.DataFrame(bert_output.loc[i,"features"]) # Get the BERT embeddings for the current line in the data file
        for j in range(2,len(features)):  # Iterate over the BERT tokens for the current line; we skip over the first 2 tokens, which don't correspond to words
            token = features.loc[j,"token"]

            # See if the character count until the current token matches the offset of any of the 3 target words
            if count_chars  == P_offset: 
                # print(token)
                emb_P += np.array(features.loc[j,"layers"][0]['values'])
                cnt_P += 1
            if count_chars in range(A_offset, A_offset + A_length): 
                # print(token)
                emb_A += np.array(features.loc[j,"layers"][0]['values'])
                cnt_A +=1
            if count_chars in range(B_offset, B_offset + B_length): 
                # print(token)
                emb_B += np.array(features.loc[j,"layers"][0]['values'])
                cnt_B +=1								
            # Update the character count
            count_chars += count_length_no_special(token)
        # Taking the average between tokens in the span of A or B, so divide the current value by the count	
        if cnt_A > 0:
            emb_A /= cnt_A   
        if cnt_B > 0:
            emb_B /= cnt_B

        # Work out the label of the current piece of text
        label = "Neither"
        if (data.loc[i,"A-coref"] == True):
            label = "A"
        if (data.loc[i,"B-coref"] == True):
            label = "B"

        # Put everything together in emb
        emb.iloc[i] = [emb_A, emb_B, emb_P, label]

    return emb

In [19]:
import time 

print("Started at ", time.ctime())
test_data = pd.read_csv("gap-test.tsv", sep = '\t')
test_emb = run_bert(test_data)
test_emb.to_json("contextual_embeddings_gap_test.json", orient = 'columns')

validation_data = pd.read_csv("gap-validation.tsv", sep = '\t')
validation_emb = run_bert(validation_data)
validation_emb.to_json("contextual_embeddings_gap_validation.json", orient = 'columns')

development_data = pd.read_csv("gap-development.tsv", sep = '\t')
development_emb = run_bert(development_data)
development_emb.to_json("contextual_embeddings_gap_development.json", orient = 'columns')
print("Finished at ", time.ctime())

Started at  Sat Aug  3 11:57:34 2019


/Users/Benjamin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:74: RuntimeWarning: invalid value encountered in true_divide
/Users/Benjamin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:73: RuntimeWarning: invalid value encountered in true_divide


Finished at  Sat Aug  3 13:29:22 2019


In [22]:
development_data.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL
ID,,,,,,,,,,,
0,development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,True,Pauline,207,False,http://en.wikipedia.org/wiki/List_of_Teachers_...
1,development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,True,Bernard Leach,251,False,http://en.wikipedia.org/wiki/Warren_MacKenzie
2,development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,False,De la Sota,246,True,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_...
3,development-4,The current members of Crime have also perform...,his,321,Hell,174,False,Henry Rosenthal,336,True,http://en.wikipedia.org/wiki/Crime_(band)
4,development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,False,Rivera,294,True,http://en.wikipedia.org/wiki/Jessica_Rivera


In [84]:
len(development_emb)
development_emb.head()

2000

,emb_A,emb_B,emb_P,label
ID,,,,
0,"[0.7388775000000001, -0.22716449999999996, 0.1...","[0.385269, 0.024027999999999997, 0.90755000000...","[-0.008841, -0.9525579999999999, 0.113496, -0....",A
1,"[0.834177, -0.47442199999999995, 1.149044, 0.1...","[-0.0622775, 0.21735149999999998, -0.366814, -...","[-0.316778, 0.9291269999999999, -0.61344599999...",A
2,"[0.016817499999999985, -0.47463500000000003, 0...","[-0.2066525, -0.58868, 0.57271025, 0.266884, 0...","[-0.6279279999999999, -0.401219, -0.140529, 0....",B
3,"[0.35994299999999996, 0.569445, -0.094911, 0.1...","[-0.1316495, -0.06583699999999999, 0.313922, -...","[0.19959000000000002, 0.226825, -0.291493, 0.2...",B
4,"[-0.192178, -0.22681325000000002, 0.1195679999...","[0.293198, -0.609892, -0.09239000000000001, -0...","[-0.011042999999999999, -1.187912, -0.898381, ...",B


In [58]:
print(development_emb.loc[0,'emb_A'].size, validation_emb.loc[0,'emb_A'].size, test_emb.loc[0,'emb_A'].size)

768 768 768


**Model (mutli-layer perceptron) and training**

In [35]:
from keras import backend, models, layers, initializers, regularizers, constraints, optimizers
from keras import callbacks as kc
from keras import optimizers as ko

from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import log_loss
import time

# Parameters of the model

dense_layer_sizes = [64,32]
dropout_rate = 0.5
learning_rate = 0.001
n_fold = 5
batch_size = 32
epochs = 1000
patience = 100
# n_test = 100
lambd = 0.1 # L2 regularization

In [28]:
#from keras.models import Sequential
from keras.layers import Input, Dense, BatchNormalization, Dropout, Activation

In [29]:
def build_mlp_model(input_shape):
    X_input = Input(input_shape)
    X = X_input
    for i in range(1,len(dense_layer_sizes)):
        X = Dense(dense_layer_sizes[i])(X)
        X = BatchNormalization()(X)
        X = Activation('relu')(X)
        X = Dropout(dropout_rate, seed=11)(X)
        
    X = Dense(3, name='output', kernel_regularizer=regularizers.l2(lambd))(X)
    X = Activation('softmax')(X)
    
    model = models.Model(inputs = X_input, outputs = X, name = "clf_model")
    return model

In [18]:
def parse_json(embeddings):
	'''
	Parses the embeddigns given by BERT, and suitably formats them to be passed to the MLP model

	Input: embeddings, a DataFrame containing contextual embeddings from BERT, as well as the labels for the classification problem
	columns: "emb_A": contextual embedding for the word A
	         "emb_B": contextual embedding for the word B
	         "emb_P": contextual embedding for the pronoun
	         "label": the answer to the coreference problem: "A", "B" or "NEITHER"

	Output: X, a numpy array containing, for each line in the GAP file, the concatenation of the embeddings of the target words
	        Y, a numpy array containing, for each line in the GAP file, the one-hot encoded answer to the coreference problem
	'''
	embeddings.sort_index(inplace = True) # Sorting the DataFrame, because reading from the json file messed with the order
	X = np.zeros((len(embeddings),3*768))
	Y = np.zeros((len(embeddings), 3))

	# Concatenate features
	for i in range(len(embeddings)):
		A = np.array(embeddings.loc[i,"emb_A"])
		B = np.array(embeddings.loc[i,"emb_B"])
		P = np.array(embeddings.loc[i,"emb_P"])
		X[i] = np.concatenate((A,B,P))

	# One-hot encoding for labels
	for i in range(len(embeddings)):
		label = embeddings.loc[i,"label"]
		if label == "A":
			Y[i,0] = 1
		elif label == "B":
			Y[i,1] = 1
		else:
			Y[i,2] = 1

	return X, Y

In [12]:
# Read development embeddigns from json file - this is the output of Bert
development = pd.read_json("contextual_embeddings_gap_development.json")
X_development, Y_development = parse_json(development)   # This will be used for testing

validation = pd.read_json("contextual_embeddings_gap_validation.json")
X_validation, Y_validation = parse_json(validation)   ## This will be used for training

test = pd.read_json("contextual_embeddings_gap_test.json")
X_test, Y_test = parse_json(test)   ## This will be used for training

In [13]:
# There may be a few NaN values, where the offset of a target word is greater than the max_seq_length of BERT.
# They are very few, so I'm just dropping the rows.
remove_test = [row for row in range(len(X_test)) if np.sum(np.isnan(X_test[row]))]
X_test = np.delete(X_test, remove_test, 0)
Y_test = np.delete(Y_test, remove_test, 0)

remove_validation = [row for row in range(len(X_validation)) if np.sum(np.isnan(X_validation[row]))]
X_validation = np.delete(X_validation, remove_validation, 0)
Y_validation = np.delete(Y_validation, remove_validation, 0)

# We want predictions for all development rows. So instead of removing rows, make them 0
remove_development = [row for row in range(len(X_development)) if np.sum(np.isnan(X_development[row]))]
X_development[remove_development] = np.zeros(3*768)

In [14]:
# Will train on data from the gap-test and gap-validation files, in total 2454 rows
X_train = np.concatenate((X_test, X_validation), axis = 0)
Y_train = np.concatenate((Y_test, Y_validation), axis = 0)

# Will predict probabilities for data from the gap-development file; initializing the predictions
prediction = np.zeros((len(X_development),3)) # testing predictions

In [36]:
# Training and cross-validation
folds = KFold(n_splits=n_fold, shuffle=True, random_state=11)
scores = []
for fold_n, (train_index, valid_index) in enumerate(folds.split(X_train)):
	# split training and validation data
	print('Fold', fold_n, 'started at', time.ctime())
	X_tr, X_val = X_train[train_index], X_train[valid_index]
	Y_tr, Y_val = Y_train[train_index], Y_train[valid_index]

	# Define the model, re-initializing for each fold
	classif_model = build_mlp_model([X_train.shape[1]])
	classif_model.compile(optimizer = optimizers.Adam(lr = learning_rate), loss = "categorical_crossentropy")
	callbacks = [kc.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights = True)]

	# train the model
	classif_model.fit(x = X_tr, y = Y_tr, epochs = epochs, batch_size = batch_size, 
                      callbacks = callbacks, validation_data = (X_val, Y_val), verbose = 0)

	# make predictions on validation and test data
	pred_valid = classif_model.predict(x = X_val, verbose = 0)
	pred = classif_model.predict(x = X_development, verbose = 0)

	# oof[valid_index] = pred_valid.reshape(-1,)
	scores.append(log_loss(Y_val, pred_valid))
	prediction += pred
prediction /= n_fold

# Print CV scores, as well as score on the test data
print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
print(scores)
print("Test score:", log_loss(Y_development,prediction))

Fold 0 started at Wed Aug  7 14:27:38 2019
Fold 1 started at Wed Aug  7 14:28:08 2019
Fold 2 started at Wed Aug  7 14:28:38 2019
Fold 3 started at Wed Aug  7 14:29:09 2019
Fold 4 started at Wed Aug  7 14:29:38 2019
CV mean score: 0.5767, std: 0.0276.
[0.5876452079185762, 0.5256018349161095, 0.5831299087374334, 0.5785930949987901, 0.6087202651756436]
Test score: 0.5320775842295536


In [37]:
# Write the prediction to file for submission at stage 1
submission = pd.read_csv("input/sample_submission_stage_1.csv", index_col = "ID")
submission["A"] = prediction[:,0]
submission["B"] = prediction[:,1]
submission["NEITHER"] = prediction[:,2]
submission.to_csv("submission_bert_stage_1.csv")

In [14]:
# Stage 2 test data
stage_2_data = pd.read_csv("input/test_stage_2.tsv", sep = '\t')

In [15]:
def run_bert_no_labels(data):

    text = data["Text"]
    text.to_csv("input.txt", index = False, header = False)

    os.system("python3 extract_features.py \
      --input_file=input.txt \
      --output_file=output.jsonl \
      --vocab_file=uncased_L-12_H-768_A-12/vocab.txt \
      --bert_config_file=uncased_L-12_H-768_A-12/bert_config.json \
      --init_checkpoint=uncased_L-12_H-768_A-12/bert_model.ckpt \
      --layers=-1 \
      --max_seq_length=256 \
      --batch_size=8")

    bert_output = pd.read_json("output.jsonl", lines = True)

    os.system("rm output.jsonl")
    os.system("rm input.txt")

    index = data.index
    columns = ["emb_A", "emb_B", "emb_P", "label"]
    emb = pd.DataFrame(index = index, columns = columns)
    emb.index.name = "ID"

    for i in range(len(data)): 
        P = data.loc[i,"Pronoun"].lower()
        A = data.loc[i,"A"].lower()
        B = data.loc[i,"B"].lower()

        P_offset = compute_offset_no_spaces(data.loc[i,"Text"], data.loc[i,"Pronoun-offset"])
        A_offset = compute_offset_no_spaces(data.loc[i,"Text"], data.loc[i,"A-offset"])
        B_offset = compute_offset_no_spaces(data.loc[i,"Text"], data.loc[i,"B-offset"])

        A_length = count_length_no_special(A)
        B_length = count_length_no_special(B)

        emb_A = np.zeros(768)
        emb_B = np.zeros(768)
        emb_P = np.zeros(768)

        # Initialize counts
        count_chars = 0
        cnt_A, cnt_B, cnt_P = 0, 0, 0

        features = pd.DataFrame(bert_output.loc[i,"features"]) 
        for j in range(2,len(features)): 
            token = features.loc[j,"token"]
            
            if count_chars  == P_offset: 
                emb_P += np.array(features.loc[j,"layers"][0]['values'])
                cnt_P += 1
            if count_chars in range(A_offset, A_offset + A_length): 
                emb_A += np.array(features.loc[j,"layers"][0]['values'])
                cnt_A +=1
            if count_chars in range(B_offset, B_offset + B_length): 
                emb_B += np.array(features.loc[j,"layers"][0]['values'])
                cnt_B +=1								
            count_chars += count_length_no_special(token)
        if cnt_A > 0:
            emb_A /= cnt_A   
        if cnt_B > 0:
            emb_B /= cnt_B

        emb.iloc[i] = [emb_A, emb_B, emb_P]

    return emb

In [ ]:
# Takes too long!
print("Started at ", time.ctime())
stage_2_emb = run_bert_no_labels(stage_2_data)
print("Finished at ", time.ctime())
stage_2_emb.to_json("contextual_embeddings_stage_2_test.json", orient = 'columns')

Unfortunately, the stage 2 data is to big to be processed on my computer.